In [2]:
%pip install pandas
%pip install ace_tools

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [3]:
%pip install math

Defaulting to user installation because normal site-packages is not writeable
ERROR: Could not find a version that satisfies the requirement math (from versions: none)
ERROR: No matching distribution found for math
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [ ]:
import cv2
import mediapipe as mp
import pandas as pd
import numpy as np
import glob
import time
from collections import deque

# 각도 계산 함수
def calculate_angle(a, b, c):
    a, b, c = np.array(a), np.array(b), np.array(c)
    ba, bc = a - b, c - b
    cosine_angle = np.clip(np.dot(ba, bc) / (np.linalg.norm(ba) * np.linalg.norm(bc)), -1.0, 1.0)
    return np.degrees(np.arccos(cosine_angle))

# CSV 파일 읽기
file_pattern = "push_csv_data/push_*.csv"
files = glob.glob(file_pattern)

# 데이터 병합
all_data = pd.concat([pd.read_csv(file) for file in files], ignore_index=True) if files else pd.DataFrame()

# 기준 각도 계산 (전체 데이터의 평균 및 범위 설정)
elbow_mean_angle = all_data['elbow_angle'].mean() if 'elbow_angle' in all_data else 90
hip_mean_angle = all_data['hip_angle'].mean() if 'hip_angle' in all_data else 170
ELBOW_MIN, ELBOW_MAX = elbow_mean_angle - 15, elbow_mean_angle + 15
HIP_MIN, HIP_MAX = hip_mean_angle - 10, hip_mean_angle + 10
ARMPIT_MIN, ARMPIT_MAX = 40, 50 #겨드랑이 각도는 기존 고정값 사용

#기준 각도(평균값) 출력
print("데이터를 기반으로 한 각도 값 범위:")
print(f"팔꿈치 기준 각도: {ELBOW_MIN:.2f}° ~ {ELBOW_MAX:.2f}°")
print(f"엉덩이 기준 각도: {HIP_MIN:.2f}° ~ {HIP_MAX:.2f}°")
print(f"겨드랑이 기준 각도: {ARMPIT_MIN:.2f}° ~ {ARMPIT_MAX:.2f}°")

# Mediapipe 초기화
mp_pose = mp.solutions.pose
pose = mp_pose.Pose()
mp_drawing = mp.solutions.drawing_utils

# 상태 지속 시간을 추적하기 위한 딕셔너리
status_duration = {"elbow": 0, "hip": 0, "armpit": 0} #지속 시간을 초 단위로 저장, 초기값은 모두 0으로 설정
status_threshold = 1.5  # 올바른 자세가 1.5초 이상 지속되면 유지됨으로 인식

# 관절의 각도 변화 기록
angle_history = {"elbow": deque(maxlen=5), "hip": deque(maxlen=5), "armpit": deque(maxlen=5)}

# 웹캠 열기
cap = cv2.VideoCapture(0)

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        print("Cannot open webcam.")
        break

    #  Mediapipe로 관절 추출
    image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = pose.process(image)
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

    if results.pose_landmarks:
        landmarks = results.pose_landmarks.landmark

        def get_point(landmark): #관절 위치 계산
            return [landmarks[landmark].x * frame.shape[1], landmarks[landmark].y * frame.shape[0]]

        try:
            shoulder = get_point(mp_pose.PoseLandmark.LEFT_SHOULDER) #왼쪽 어깨 위치 계산
            hip = get_point(mp_pose.PoseLandmark.LEFT_HIP) #왼쪽 엉덩이 위치 계산

            # 시각적 가이드라인 및 점 그리기
            cv2.line(image, tuple(map(int, shoulder)), tuple(map(int, hip)), (255, 0, 0), 2) #어깨와 엉덩이 사이 선
            cv2.circle(image, tuple(map(int, shoulder)), 5, (0, 255, 0), -1) #어깨에 원 그리기
            cv2.circle(image, tuple(map(int, hip)), 5, (0, 255, 0), -1) #엉덩이에 원 그리기

            # 각도 계산
            elbow = get_point(mp_pose.PoseLandmark.LEFT_ELBOW) #왼쪽 팔꿈치 위치 계산
            wrist = get_point(mp_pose.PoseLandmark.LEFT_WRIST) #왼족 손목 위치 계산
            knee = get_point(mp_pose.PoseLandmark.LEFT_KNEE) #왼족 무릎 위치 계산

            elbow_angle = calculate_angle(shoulder, elbow, wrist) #팔꿈치 각도 계산
            hip_angle = calculate_angle(shoulder, hip, knee) #엉덩이 각도 계산
            armpit_angle = calculate_angle(elbow, shoulder, hip) #겨드랑이 각도 계산
            
            # 기준 각도 출력
            print(f"팔꿈치 기준 각도: {ELBOW_MIN:.2f}° ~ {ELBOW_MAX:.2f}°")
            print(f"엉덩이 기준 각도: {HIP_MIN:.2f}° ~ {HIP_MAX:.2f}°")
            print(f"겨드랑이 기준 각도: {ARMPIT_MIN:.2f}° ~ {ARMPIT_MAX:.2f}°")
                        

            # 각도 기록
            angle_history["elbow"].append(elbow_angle)
            angle_history["hip"].append(hip_angle)
            angle_history["armpit"].append(armpit_angle)

            # 평균 각도 계산
            avg_elbow_angle = np.mean(angle_history["elbow"]) #팔꿈치의 평균 각도 계산
            avg_hip_angle = np.mean(angle_history["hip"]) #엉덩이의 평균 각도 계산
            avg_armpit_angle = np.mean(angle_history["armpit"]) #겨드랑이의 평균 각도 계산

            # 업데이트
            current_time = time.time() #현재 시간 기록

            # 각도 범위 
            if ELBOW_MIN <= avg_elbow_angle <= ELBOW_MAX:
                status_duration["elbow"] = current_time
            if HIP_MIN <= avg_hip_angle <= HIP_MAX:
                status_duration["hip"] = current_time
            if ARMPIT_MIN <= avg_armpit_angle <= ARMPIT_MAX:
                status_duration["armpit"] = current_time

            # 피드백 생성
            feedback = []
            if current_time - status_duration["elbow"] > status_threshold:
                feedback.append("Incorrect elbow angle") #팔꿈치 각도가 부정확한 경우 피드백 추가
            if current_time - status_duration["hip"] > status_threshold:
                feedback.append("Incorrect hip angle") #엉덩이 각도가 부정확한 경우 피드백 추가
            if current_time - status_duration["armpit"] > status_threshold:
                feedback.append("Incorrect armpit angle") #겨드랑이 각도가 부정확한 경우 피드백 추가

            # 피드백 텍스트 표시
            feedback_text = "\n".join(feedback) if feedback else "Correct posture"

            # 피드백을 이미지에 표시
            y_offset = 20
            for i, line in enumerate(feedback_text.split("\n")):
                color = (0, 255, 0) if "Correct posture" in line else (0, 0, 255) #상태에 따른 색상 설정
                cv2.putText(image, line, (10, y_offset + i * 20), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 1)

            # 현재 각도 정보 표시
            cv2.putText(image, f"Elbow: {avg_elbow_angle:.2f}°", (10, y_offset + 120), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 1)
            cv2.putText(image, f"Hip: {avg_hip_angle:.2f}°", (10, y_offset + 140), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 1)
            cv2.putText(image, f"Armpit: {avg_armpit_angle:.2f}°", (10, y_offset + 160), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 1)

        except Exception as e:
            print(f"Error occurred: {e}")

        # Mediapipe 시각화
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)

    # 이미지 표시
    cv2.imshow("Push-up Analysis", image)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()
cv2.waitKey()


팔꿈치 기준 각도: 75.00° ~ 105.00°
엉덩이 기준 각도: 160.00° ~ 180.00°
겨드랑이 기준 각도: 40.00° ~ 50.00°


I0000 00:00:1737518990.353579 8764578 gl_context.cc:369] GL version: 2.1 (2.1 ATI-6.1.13), renderer: AMD Radeon Pro 555X OpenGL Engine
INFO: Created TensorFlow Lite XNNPACK delegate for CPU.
W0000 00:00:1737518990.653252 8794281 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1737518990.693357 8794281 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1737518991.658224 8794280 landmark_projection_calculator.cc:186] Using NORM_RECT without IMAGE_DIMENSIONS is only supported for the square ROI. Provide IMAGE_DIMENSIONS or use PROJECTION_MATRIX.
2025-01-22 13:09:52.418 Python[98755:8764578] +[IMKClient subclass]: chose IMKClient_Modern
2025-01-22 13:09:52.418 Python[98755:8764578] +[IMKInputSession subclass]: chose IMKInputSession_Modern
